07/11/22

Salazar Vega Rodrigo


In [7]:
import os 
from bs4 import BeautifulSoup
import re
import nltk
import numpy as np
from nltk.tokenize import word_tokenize

In [2]:
# Paths of importance
stopwords_path = r'C:\Users\hp\Documents\VSCode\PNL-Escom\ss\stopwords_and_lemmas\stopwords_es.txt'
lemmas_path = r'D:\rod_e\Documents\VSCode\PNL-Escom\Em Computation\stopwords_and_lemmas\generate.txt'

Funciones para la limpieza de texto

In [3]:
def normalizar(text):
    # nltk.download('stopwords') 
    '''
    Funcion para normalizar el texto y eliminar stopwords
    text : texto para normalizar
    '''
  
    from nltk.corpus import stopwords 
    stop_words = set(stopwords.words('spanish'))
    lower_string = text.lower()

    no_number_string = re.sub(r'\d+','',lower_string) 
    no_punc_string = re.sub(r'[^\w\s]','', no_number_string)  
    no_wspace_string = no_punc_string.strip() 
    # no_wspace_string 
    
    lst_string = [no_wspace_string][0].split() 
    # print(lst_string)
    no_stpwords_string="" 
    for i in lst_string: 
        if not i in stop_words: 
            no_stpwords_string += i+' '
            
    no_stpwords_string = no_stpwords_string[:-1]
    
    return no_stpwords_string

def lemmatize(text, path = lemmas_path):
    """
        Here you can lemmatize your words.
        words: your words free of stop words.
        path: the path where are the lemmas you want.
    """
    lemmas = dict()
    with open(path, encoding = 'latin-1') as file:
        lines = file.readlines()
        lines = [w.strip() for w in lines]
        for line in lines:
            line = line.strip()
            if line != '':
                words = line.split()
                token = words[0].strip()
                token = token.replace('#', '')
                lemma = words[-1].strip()
                lemmas[token] = lemma
    lemmatized_text = list()
    for word in text:
        if word in lemmas.keys():
            lemmatized_text.append(lemmas[word])
        else:
            lemmatized_text.append(word)
    return lemmatized_text


def get_vocabulary(words):
    """
        Here you can get the vocabulary of your words.
        words: list of words.
    """
    vocabulary = list(sorted(set(words)))
    return vocabulary

#### Funciones de extraccion de titulos
 

In [4]:
def splitIntoArticules(path, fname):
    with open(path + fname, 'r', encoding='utf-8') as file:
        text = file.read()
    text.replace(u'\x97', '')

    articles = re.split('<h3>', text)
    
    arts = []
    for article in articles:
        soup = BeautifulSoup(article, 'lxml')
        text = soup.get_text()
        text.replace(u'\x97', '')
        arts.append(text)
    
    return arts

Funciones para obtener el conteo de cada palabra y la probabilidad en el primer texto

In [160]:
def conteo_palabras(titles = list()):
    """
    Funcion para obtener el conteo de palabras dentro de los titulos del archivo que previamente se ha lematizado y normalizado, así como obtener las probabilidades de cada palabra en todo el texto
    tilles : titulos del archivo ya normalizados y lematizados
    """
    word_frec = dict()
    
    for title in titles:
        tokens = word_tokenize(title)
        for token in tokens:
            if token not in word_frec.keys():
                word_frec[token] = 1
            else:
                word_frec[token] +=1

    # word_frec = (sorted(word_frec.items(), key = lambda item: item[1], reverse = True))
    return word_frec      

def obtener_probabilidad(titles = list()):
    '''
    Funcion para obtener las probabilidades de las palabrasa que contienen los titulos
    titles = lista que contiene el texto del que quieres obtener las probabilidades
    '''
    
    word_prob = dict()
    total = 0
    llaves = []
    pr = []

    for title in titles:
        tokens = word_tokenize(title)
        for token in tokens:
            total += 1
            if token not in word_prob.keys():
                word_prob[token] = 1
            else:
                word_prob[token] +=1

    for v,j in word_prob.items():
        llaves.append(v)
        pr.append(j)
    vector = np.array(pr)
    vector = vector / total

    word_prb = dict()
    for i in range(len(vector)):
        word_prb[str(llaves[i])] = vector[i]
        
    return word_prb    

Obtenemos los titulos del archivo e960401_mod.html

In [161]:
# Obtenemos y normalizamos el texto del primer archivo
artic = splitIntoArticules(path='./EXCELSIOR_100_files/', fname='e960401_mod.htm')
art_norm = [normalizar(artic[i]) for i in range(len(artic))]
art_lemma = lemmatize(art_norm, './nlp_functions/stopwords_and_lemmas/generate.txt')
# art_vocabulary = get_vocabulary(art_lemma)
conteo = conteo_palabras(art_lemma)
prob = obtener_probabilidad(art_lemma)


In [167]:
art_voc = get_vocabulary(art_lemma)
voc_conteo = conteo_palabras(art_voc)
voc_prob = obtener_probabilidad(art_voc)

In [171]:
valores = [i for i in voc_prob.values()]

In [166]:
artic = splitIntoArticules(path='./EXCELSIOR_100_files/', fname='e960401_mod.htm')
